# Project - Customer Support chatbot for a restaurant

In [16]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI 
import gradio as gr 

# 1- Initialization

In [8]:
load_dotenv()

open_api_key = os.getenv("OPENAI_API_KEY")
if open_api_key:
    print(f"OpenAI API Key exists and begins {open_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [6]:
system_message = "You are a helpful assistant for a Restaurant called SavoryAI." 
system_message += "Give short, courteous answers, no more than 1 sentence." 
system_message += "Always be accurate. If you don’t know the answer, say so."

# 2- Interacting with OpenAI 

In [9]:
def chat(message, history):
    messages = [
        {"role": "system", "content": system_message}
    ] + history + [
        {"role": "user", "content": message}
    ]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


# 3- Tools

## Pick a Customer-Support “Tool”

In this project, our Chatbot would perform "Check an order’s shipping status". That’s our new tool.

In [10]:
orders = {
    "1234": "In Transit",
    "5678": "Delivered",
    "9999": "Pending"
}

def get_order_status(order_id):
    print(f"Tool get_order_status called for {order_id}")
    return orders.get(order_id, "No record found")

In [11]:
# Create the function schema (for “tools”)
# Note: it is always good idea to give an example in "description".
order_status_function = {
    "name": "get_order_status",
    "description": (
        "Check the shipping status of a customer's order by ID. "
        "Call this whenever you need to find out a package's status. "
        "For example, when a user says 'Where is my order #1234?' "
        "or 'Has my package #5678 shipped yet?'."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The order ID provided by the customer"
            }
        },
        "required": ["order_id"],
        "additionalProperties": False
    }
}

In [14]:
# And this is included in a list of tools:
tools = [{"type": "function", "function": order_status_function}]

# 4- Getting OpenAI to use our Tool

In [15]:
def chat(message, history):
    messages = [
        {"role": "system", "content": system_message}
    ] + history + [
        {"role": "user", "content": message}
    ]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    if response.choices[0].finish_reason == "tool_calls":
        # LLM wants to use a tool
        message = response.choices[0].message
        response, some_value = handle_tool_call(message)
        messages.append(message)
        messages.append(response)

        # Then call the model again to provide final text 
        # (the AI might want to reference the tool's output in its final answer)
        response = openai.chat.completions.create(
            model=MODEL, 
            messages=messages
        )
    return response.choices[0].message.content

In [22]:
def handle_tool_call(message):
    # 'tool_calls' is a list of function calls from the LLM
    tool_call = message.tool_calls[0]
    tool_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if tool_name == "get_order_status":
        order_id = arguments.get("order_id")
        status = get_order_status(order_id)
        return {
            "role": "tool",
            "content": json.dumps({"order_id": order_id, "status": status}),
            "tool_call_id": tool_call.id
        }, order_id
    

In [23]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Tool get_order_status called for 1234
Tool get_order_status called for 546
Tool get_order_status called for 5678
